In [1]:
using LinearAlgebra
P = [ 1.0  0.0  1.0  2.0
     -1.0  0.0  1.0  2.0
      0.0  1.0  0.0 -3.0
      0.0 -1.0  0.0 -3.0
      0.0  0.0 -2.0  2.0]
for i in 1:4
    P[:, i] /= norm(P[:, i])
end
@show P' * ones(5)
round.(P' * P, digits=16)

P' * ones(5) = [0.0, 0.0, 0.0, 0.0]


4×4 Array{Float64,2}:
 1.0   0.0   0.0   0.0
 0.0   1.0   0.0  -0.0
 0.0   0.0   1.0  -0.0
 0.0  -0.0  -0.0   1.0

In [2]:
using HybridSystems
H1 = [0.2 0.8 0   0   0
      0   0.2 0.8 0   0
      0   0   0.2 0.8 0
      0   0   0   0.2 0.8
      0.2 0   0   0   0.8]
A1 = P' * H1 * P
H2 = [1   0   0   0   0
      0.8 0.2 0   0   0
      0   0   1   0   0
      0   0   0.8 0.2 0
      0.2 0   0   0   1]
A2 = P' * H2 * P
H3 = [1   0   0   0   0
      0   1   0   0   0
      0   0.8 0.2 0   0
      0   0   0   1   0
      0   0   0   0.2 0.8]
A3 = P' * H3 * P
function automaton(N)
    # See [P17, Figure 2.17] for what automaton(3) should be
    a = LightAutomaton(N)
    # Node i means, H1 was used i-1 steps ago
    add_transition!(a, 1, 1, 1)
    for i in 2:N
        add_transition!(a, i-1, i, 2)
        add_transition!(a, i-1, i, 3)
        add_transition!(a, i, 1, 1)
    end
    display(a)
    return a
end
ss(N) = discreteswitchedsystem([A1, A2, A3], automaton(N))

ss (generic function with 1 method)

In [3]:
for t in transitions(automaton(2))
    @show t
end

LightAutomaton{LightGraphs.SimpleGraphs.SimpleDiGraph{Int64},LightGraphs.SimpleGraphs.SimpleEdge{Int64}}({2, 3} directed simple Int64 graph, Dict(Edge 1 => 1=>Dict(1=>1),Edge 1 => 2=>Dict(2=>2,3=>3),Edge 2 => 1=>Dict(4=>1)), 4, 4)

t = HybridSystems.LightTransition{LightGraphs.SimpleGraphs.SimpleEdge{Int64}}(Edge 1 => 1, 1)
t = HybridSystems.LightTransition{LightGraphs.SimpleGraphs.SimpleEdge{Int64}}(Edge 1 => 2, 2)
t = HybridSystems.LightTransition{LightGraphs.SimpleGraphs.SimpleEdge{Int64}}(Edge 1 => 2, 3)
t = HybridSystems.LightTransition{LightGraphs.SimpleGraphs.SimpleEdge{Int64}}(Edge 2 => 1, 4)


In [4]:
using SwitchOnSafety
ρ(A1)

0.7273364816948462

In [5]:
using MathOptInterfaceMosek
using JuMP
factory = with_optimizer(MosekOptimizer, QUIET=true);

In [6]:
using SwitchOnSafety
soslyapb(ss(1), 1; factory=factory, tol=1e-7, verbose=1)

LightAutomaton{LightGraphs.SimpleGraphs.SimpleDiGraph{Int64},LightGraphs.SimpleGraphs.SimpleEdge{Int64}}({1, 1} directed simple Int64 graph, Dict(Edge 1 => 1=>Dict(1=>1)), 1, 1)

Lower bound: 0.7273364445083259
Upper bound: 0.7273364878609576
Log-diff   : 5.960464477539063e-8 ≤ 1.0e-7


(0.7273364445083259, 0.7273364878609576)

In [12]:
using SwitchOnSafety
soslyapb(ss(2), 1; factory=factory, tol=1e-5, verbose=1)

LightAutomaton{LightGraphs.SimpleGraphs.SimpleDiGraph{Int64},LightGraphs.SimpleGraphs.SimpleEdge{Int64}}({2, 3} directed simple Int64 graph, Dict(Edge 1 => 1=>Dict(1=>1),Edge 1 => 2=>Dict(2=>2,3=>3),Edge 2 => 1=>Dict(4=>1)), 4, 4)

Lower bound: 1.0120936982417008
Upper bound: 1.012101419933283
Log-diff   : 7.629394531036629e-6 ≤ 1.0e-5


(0.795658855292319, 1.012101419933283)

In [11]:
using SwitchOnSafety
soslyapb(ss(3), 1; factory=factory, tol=1e-5, verbose=1)

LightAutomaton{LightGraphs.SimpleGraphs.SimpleDiGraph{Int64},LightGraphs.SimpleGraphs.SimpleEdge{Int64}}({3, 5} directed simple Int64 graph, Dict(Edge 1 => 1=>Dict(1=>1),Edge 3 => 1=>Dict(7=>1),Edge 1 => 2=>Dict(2=>2,3=>3),Edge 2 => 3=>Dict(5=>2,6=>3),Edge 2 => 1=>Dict(4=>1)), 7, 7)

Lower bound: 1.0872521381151024
Upper bound: 1.0872604332222622
Log-diff   : 7.629394531166733e-6 ≤ 1.0e-5


(0.8016883058828722, 1.0872604332222622)

In [10]:
using SwitchOnSafety
soslyapb(ss(4), 1; factory=factory, tol=1e-5, verbose=1)

LightAutomaton{LightGraphs.SimpleGraphs.SimpleDiGraph{Int64},LightGraphs.SimpleGraphs.SimpleEdge{Int64}}({4, 7} directed simple Int64 graph, Dict(Edge 1 => 1=>Dict(1=>1),Edge 3 => 1=>Dict(7=>1),Edge 1 => 2=>Dict(2=>2,3=>3),Edge 4 => 1=>Dict(10=>1),Edge 2 => 3=>Dict(5=>2,6=>3),Edge 2 => 1=>Dict(4=>1),Edge 3 => 4=>Dict(9=>3,8=>2)), 10, 10)

Lower bound: 1.1047868046212521
Upper bound: 1.1047952335078113
Log-diff   : 7.629394531402656e-6 ≤ 1.0e-5


(0.7957457050003699, 1.1047952335078113)